## Chat Completion - Multilingual-Thinking

This sample shows how use `chat-completion` components from the `azureml` system registry to fine tune a model to complete a conversation between 2 people using Multilingual-Thinking dataset. We then deploy the fine tuned model to an online endpoint for real time inference.

### Training data
We will use the [Multilingual-Thinking](https://huggingface.co/datasets/HuggingFaceH4/Multilingual-Thinking) dataset. Multilingual-Thinking is a reasoning dataset where the chain-of-thought has been translated from English into one of 4 languages: Spanish, French, Italian, and German. The dataset was created by sampling 1k training samples from the SystemChat subset of SmolTalk2 and translating the reasoning traces with another language model.

### Model
We will use the `gpt-oss-20b` model to show how user can finetune a model for chat-completion task. 

### Outline
* Setup pre-requisites such as compute.
* Pick a model to fine tune.
* Pick and explore training data.
* Configure the fine tuning job.
* Run the fine tuning job.
* Review training and evaluation metrics. 
* Register the fine tuned model. 
* Deploy the fine tuned model for real time inference.
* Clean up resources. 

* Check or create compute. A single GPU node can have multiple GPU cards. For example, in one node of `Standard_NC96ads_A100_v4` there are 4 NVIDIA A100 GPUs, while in `Standard_ND96isr_H100_v5` there are 8 NVIDIA H100 GPUs. Refer to the [docs](https://learn.microsoft.com/en-us/azure/virtual-machines/sizes-gpu) for this information. The number of GPU cards per node is set in the param `gpus_per_node` below. Setting this value correctly will ensure utilization of all GPUs in the node. The recommended GPU compute SKUs can be found [here](https://learn.microsoft.com/en-us/azure/virtual-machines/ncv3-series), [here](https://learn.microsoft.com/en-us/azure/virtual-machines/ndv2-series), and [here](https://learn.microsoft.com/en-us/azure/virtual-machines/ndv5-series).


Install dependencies by running below cell. This is not an optional step if running in a new environment.

In [ ]:
%pip install azure-ai-ml
%pip install azure-identity
%pip install datasets==4.0.0
%pip install mlflow
%pip install azureml-mlflow

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

try:
    workspace_ml_client = MLClient.from_config(credential=credential)
except:
    workspace_ml_client = MLClient(
        credential,
        subscription_id="",
        resource_group_name="",
        workspace_name="",
    )

# the models, fine tuning pipelines and environments are available in the AzureML registry, "azureml-openai-oss"
registry_ml_client = MLClient(credential, registry_name="azureml-openai-oss")
experiment_name = "chat_completion_gpt_oss"

# generating a unique timestamp that can be used for names and versions that need to be unique
timestamp = str(int(time.time()))

### 2. Pick gpt-oss model to fine tune

`gpt-oss-20b` is a open source model from OpenAI for lower latency, and local or specialized use cases (21B parameters with 3.6B active parameters), we need to finetune the model for our specific purpose in order to use it. You can browse these models in the Model Catalog in the AzureML Studio, filtering by the `chat-completion` task. 

Note the model id property of the model. This will be passed as input to the fine tuning job. This is also available as the `Asset ID` field in model details page in AzureML Studio Model Catalog. 

In [ ]:
model_name = "gpt-oss-20b"
foundation_model = registry_ml_client.models.get(model_name, version="6")
print(
    "\n\nUsing model name: {0}, version: {1}, id: {2} for fine tuning".format(
        foundation_model.name, foundation_model.version, foundation_model.id
    )
)

### 3. Create a compute to be used with the job

The finetune job works `ONLY` with `GPU` compute. The size of the compute depends on how big the model is and in most cases it becomes tricky to identify the right compute for the job. In this cell, we guide the user to select the right compute for the job.

`NOTE1` The computes listed below work with the most optimized configuration. Any changes to the configuration might lead to Cuda Out Of Memory error. In such cases, try to upgrade the compute to a bigger compute size.

`NOTE2` While selecting the compute_cluster_size below, make sure the compute is available in your resource group. If a particular compute is not available you can make a request to get access to the compute resources.

In [ ]:
import ast

if "finetune_compute_allow_list" in foundation_model.tags:
    computes_allow_list = ast.literal_eval(
        foundation_model.tags["finetune_compute_allow_list"]
    )  # convert string to python list
    print(f"Please create a compute from the above list - {computes_allow_list}")
else:
    computes_allow_list = None
    print("`finetune_compute_allow_list` is not part of model tags")

In [ ]:
# If you have a specific compute size to work with change it here. By default we use the 8 x V100 compute from the above list
compute_cluster_size = "Standard_NC96ads_A100_v4"

# If you already have a gpu cluster, mention it here. Else will create a new one with the name 'gpu-a100-nc-series'
compute_cluster = "gpu-a100-nc-series"

try:
    compute = workspace_ml_client.compute.get(compute_cluster)
    print("The compute cluster already exists! Reusing it for the current run")
except Exception as ex:
    print(
        f"Looks like the compute cluster doesn't exist. Creating a new one with compute size {compute_cluster_size}!"
    )
    try:
        print("Attempt #1 - Trying to create a dedicated compute")
        compute = AmlCompute(
            name=compute_cluster,
            size=compute_cluster_size,
            tier="Dedicated",
            max_instances=2,  # For multi node training set this to an integer value more than 1
        )
        workspace_ml_client.compute.begin_create_or_update(compute).wait()
    except Exception as e:
        try:
            print(
                "Attempt #2 - Trying to create a low priority compute. Since this is a low priority compute, the job could get pre-empted before completion."
            )
            compute = AmlCompute(
                name=compute_cluster,
                size=compute_cluster_size,
                tier="LowPriority",
                max_instances=2,  # For multi node training set this to an integer value more than 1
            )
            workspace_ml_client.compute.begin_create_or_update(compute).wait()
        except Exception as e:
            print(e)
            raise ValueError(
                f"WARNING! Compute size {compute_cluster_size} not available in workspace"
            )


# Sanity check on the created compute
compute = workspace_ml_client.compute.get(compute_cluster)
if compute.provisioning_state.lower() == "failed":
    raise ValueError(
        f"Provisioning failed, Compute '{compute_cluster}' is in failed state. "
        f"please try creating a different compute"
    )

if computes_allow_list is not None:
    computes_allow_list_lower_case = [x.lower() for x in computes_allow_list]
    if compute.size.lower() not in computes_allow_list_lower_case:
        raise ValueError(
            f"VM size {compute.size} is not in the allow-listed computes for finetuning"
        )
else:
    # Computes with K80 GPUs are not supported
    unsupported_gpu_vm_list = [
        "standard_nc6",
        "standard_nc12",
        "standard_nc24",
        "standard_nc24r",
    ]
    if compute.size.lower() in unsupported_gpu_vm_list:
        raise ValueError(
            f"VM size {compute.size} is currently not supported for finetuning"
        )


# This is the number of GPUs in a single node of the selected 'vm_size' compute.
# Setting this to less than the number of GPUs will result in underutilized GPUs, taking longer to train.
# Setting this to more than the number of GPUs will result in an error.
gpu_count_found = False
workspace_compute_sku_list = workspace_ml_client.compute.list_sizes()
available_sku_sizes = []
for compute_sku in workspace_compute_sku_list:
    available_sku_sizes.append(compute_sku.name)
    if compute_sku.name.lower() == compute.size.lower():
        gpus_per_node = compute_sku.gpus
        gpu_count_found = True
# if gpu_count_found not found, then print an error
if gpu_count_found:
    print(f"Number of GPU's in compute {compute.size}: {gpus_per_node}")
else:
    raise ValueError(
        f"Number of GPU's in compute {compute.size} not found. Available skus are: {available_sku_sizes}."
        f"This should not happen. Please check the selected compute cluster: {compute_cluster} and try again."
    )

### 4. Pick the dataset for fine-tuning the model

We use the [Multilingual-Thinking](https://huggingface.co/datasets/HuggingFaceH4/Multilingual-Thinking) dataset. The dataset has train split, suitable for:
* Supervised fine-tuning (sft).
* Reasoning .
**Key points:**
- Only one split is present (e.g., `train`), with 1000 examples.
- Each example contains fields such as `reasoning_language`, `developer`, `user`, `analysis`, `final`, and a list of `messages`.
- The `messages` field is a list of message objects, each with a `content`, `role`, and optionally `thinking`.
- Typical roles include: `system` (for instructions/context), `user`, and `assistant`.

> The dataset is stored in jsonl. You can load and visualize it using pandas.

#### Example data schema

```json
{
  "reasoning_language": "French",
  "developer": "You are an AI chatbot with a lively and energetic personality.",
  "user": "Can you show me the latest trends on Twitter right now?",
  "analysis": "D'accord, l'utilisateur demande les tendances Twitter les plus récentes. Tout d'abord, je dois vérifier si j'ai accès à des données en temps réel. Étant donné que je ne peux pas naviguer sur Internet ou accéder directement à l'API de Twitter, je ne peux pas fournir des tendances en direct. Cependant, je peux donner quelques conseils généraux sur la façon de les trouver.\n\nJe devrais préciser que les tendances Twitter évoluent rapidement et sont spécifiques à chaque région. Je pourrais suggérer de consulter la section « En vogue » sur l'application ou le site web. Aussi, l'utilisation de hashtags et le suivi d'utilisateurs pertinents pourraient être utiles. Il est important de souligner que les tendances varient selon la région et l'heure de la journée. Je devrais garder un ton amical et bienveillant, peut-être ajouter un emoji pour rester léger. Je vais structurer ma réponse étape par étape pour faciliter la lecture. Je dois m'excuser de ne pas pouvoir fournir des données en temps réel et proposer d'autres méthodes. Je conserverai un langage simple et convivial, en évitant les termes techniques.",
  "final": "Hey there!  While I can't check Twitter (X) in real-time or access live data, I can share some tips to help you spot the latest trends:\n\n1. **Open the \"Trending\" tab** on the Twitter app or website – it updates constantly!  \n2. **Search for hashtags** like #Trending or #Viral to see what’s blowing up.  \n3. **Follow accounts** that curate trends (e.g., @TrendingNow, @ViralThreads).  \n4. **Check regional trends** – they often differ by location!  \n\nRemember, trends are *super fast-moving* and often tied to pop culture, memes, or breaking news. For example, recent trends have included viral challenges (like the \"Distracted Boyfriend\" meme revival), celebrity drama, or unexpected events (hello, weather disasters!).  \n\nWant me to brainstorm *what* might trend next? I’ve got ideas!",
  "messages": [
    {
      "content": "reasoning language: French\n\nYou are an AI chatbot with a lively and energetic personality.",
      "role": "system",
      "thinking": null
    },
    {
      "content": "Can you show me the latest trends on Twitter right now?",
      "role": "user",
      "thinking": null
    },
    {
      "content": "Hey there!  While I can't check Twitter (X) in real-time or access live data, I can share some tips to help you spot the latest trends:\n\n1. **Open the \"Trending\" tab** on the Twitter app or website – it updates constantly!  \n2. **Search for hashtags** like #Trending or #Viral to see what’s blowing up.  \n3. **Follow accounts** that curate trends (e.g., @TrendingNow, @ViralThreads).  \n4. **Check regional trends** – they often differ by location!  \n\nRemember, trends are *super fast-moving* and often tied to pop culture, memes, or breaking news. For example, recent trends have included viral challenges (like the \"Distracted Boyfriend\" meme revival), celebrity drama, or unexpected events (hello, weather disasters!).  \n\nWant me to brainstorm *what* might trend next? I’ve got ideas!",
      "role": "assistant",
      "thinking": "D'accord, l'utilisateur demande les tendances Twitter les plus récentes. Tout d'abord, je dois vérifier si j'ai accès à des données en temps réel. Étant donné que je ne peux pas naviguer sur Internet ou accéder directement à l'API de Twitter, je ne peux pas fournir des tendances en direct. Cependant, je peux donner quelques conseils généraux sur la façon de les trouver.\n\nJe devrais préciser que les tendances Twitter évoluent rapidement et sont spécifiques à chaque région. Je pourrais suggérer de consulter la section « En vogue » sur l'application ou le site web. Aussi, l'utilisation de hashtags et le suivi d'utilisateurs pertinents pourraient être utiles. Il est important de souligner que les tendances varient selon la région et l'heure de la journée. Je devrais garder un ton amical et bienveillant, peut-être ajouter un emoji pour rester léger. Je vais structurer ma réponse étape par étape pour faciliter la lecture. Je dois m'excuser de ne pas pouvoir fournir des données en temps réel et proposer d'autres méthodes. Je conserverai un langage simple et convivial, en évitant les termes techniques."
    }
  ]
}
```

> **Note:** The `developer` and `system` roles provide custom instructions or context for the model. The `user` and `assistant` roles represent the conversation turns. The `thinking` field can be used for intermediate reasoning or analysis.


In [ ]:
import json
import random
from datasets import load_dataset


def save_hf_dataset_to_jsonl(dataset, output_file):
    """Save HuggingFace dataset to JSONL file."""
    with open(output_file, "w", encoding="utf-8") as f:
        for record in dataset:
            f.write(json.dumps(record, ensure_ascii=False, default=str) + "\n")
    print(f"Saved {len(dataset)} records to {output_file}")


def convert_multilingual_thinking(input_file, output_file):
    """Convert multilingual thinking dataset to standard JSONL format with only messages field."""
    with open(input_file, "r", encoding="utf-8") as f:
        lines = f.readlines()
    json_objects = [json.loads(line) for line in lines if line.strip()]
    with open(output_file, "w", encoding="utf-8") as f:
        for obj in json_objects:
            cleaned_messages = []
            for msg in obj.get("messages", []):
                cleaned_msg = {"role": msg["role"], "content": msg["content"]}
                if "thinking" in msg:
                    cleaned_msg["thinking"] = msg["thinking"]
                cleaned_messages.append(cleaned_msg)
            output_obj = {"messages": cleaned_messages}
            f.write(json.dumps(output_obj, ensure_ascii=False) + "\n")
    print(f"Converted {len(json_objects)} records to {output_file}")


def split_jsonl(
    input_file,
    train_file,
    valid_file,
    test_file,
    train_size=800,
    valid_size=100,
    test_size=100,
    seed=42,
):
    """Split JSONL file into train/valid/test splits."""
    with open(input_file, "r", encoding="utf-8") as f:
        lines = [line for line in f if line.strip()]
    random.seed(seed)
    random.shuffle(lines)
    train_lines = lines[:train_size]
    valid_lines = lines[train_size : train_size + valid_size]
    test_lines = lines[train_size + valid_size : train_size + valid_size + test_size]
    with open(train_file, "w", encoding="utf-8") as f:
        f.writelines(train_lines)
    with open(valid_file, "w", encoding="utf-8") as f:
        f.writelines(valid_lines)
    with open(test_file, "w", encoding="utf-8") as f:
        f.writelines(test_lines)
    print(
        f"Train: {len(train_lines)}, Valid: {len(valid_lines)}, Test: {len(test_lines)}"
    )


# Usage example:
dataset = load_dataset("HuggingFaceH4/Multilingual-Thinking", split="train")
save_hf_dataset_to_jsonl(dataset, "multilingual_thinking_raw_data.json")
convert_multilingual_thinking(
    "multilingual_thinking_raw_data.json", "multilingual_thinking_formatted_data.jsonl"
)
split_jsonl(
    "multilingual_thinking_formatted_data.jsonl",
    "multilingual_thinking_train.jsonl",
    "multilingual_thinking_valid.jsonl",
    "multilingual_thinking_test.jsonl",
)

In [ ]:
# load the ./multilingual_thinking_train.jsonl file into a pandas dataframe and show the first row
import pandas as pd

pd.set_option(
    "display.max_colwidth", 0
)  # set the max column width to 0 to display the full text
df = pd.read_json("./multilingual_thinking_train.jsonl", lines=True)
df.head(1)

### 5. Submit the fine tuning job using the the model and data as inputs
 
Create the job that uses the `chat-completion` pipeline component. [Learn more](https://github.com/Azure/azureml-assets/blob/main/assets/training/finetune_acft_hf_nlp/components/pipeline_components/chat_completion/README.md) about all the parameters supported for fine tuning.

Define finetune parameters

Finetune parameters can be grouped into 2 categories - training parameters, optimization parameters

Training parameters define the training aspects such as - 
1. the optimizer, scheduler to use
2. the metric to optimize the finetune
3. number of training steps and the batch size
and so on

Optimization parameters help in optimizing the GPU memory and effectively using the compute resources. Below are few of the parameters that belong to this category. _The optimization parameters differs for each model and are packaged with the model to handle these variations._
1. enable the deepspeed and LoRA
2. enable mixed precision training
2. enable multi-node training 

Note: Supervised finetuning may result in loosing alignment or catastrophic forgetting. We recommend checking for this issue and running an alignment stage after you finetune.

In [ ]:
# Default training parameters
training_parameters = dict(
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=5e-6,
    lr_scheduler_type="cosine",
)
# Default optimization parameters
optimization_parameters = dict(
    apply_lora="true",
    apply_deepspeed="true",
    deepspeed_stage=3,
)
# Let's construct finetuning parameters using training and optimization paramters.
finetune_parameters = {**training_parameters, **optimization_parameters}

# Each model finetuning works best with certain finetuning parameters which are packed with model as `model_specific_defaults`.
# Let's override the finetune_parameters in case the model has some custom defaults.
if "model_specific_defaults" in foundation_model.tags:
    print("Warning! Model specific defaults exist. The defaults could be overridden.")
    finetune_parameters.update(
        ast.literal_eval(  # convert string to python dict
            foundation_model.tags["model_specific_defaults"]
        )
    )
print(
    f"The following finetune parameters are going to be set for the run: {finetune_parameters}"
)

In [ ]:
# Set the pipeline display name for distinguishing different runs from the name
def get_pipeline_display_name():
    batch_size = (
        int(finetune_parameters.get("per_device_train_batch_size", 1))
        * int(finetune_parameters.get("gradient_accumulation_steps", 1))
        * int(gpus_per_node)
        * int(finetune_parameters.get("num_nodes_finetune", 1))
    )
    scheduler = finetune_parameters.get("lr_scheduler_type", "linear")
    deepspeed = finetune_parameters.get("apply_deepspeed", "false")
    ds_stage = finetune_parameters.get("deepspeed_stage", "2")
    if deepspeed == "true":
        ds_string = f"ds{ds_stage}"
    else:
        ds_string = "nods"
    lora = finetune_parameters.get("apply_lora", "false")
    if lora == "true":
        lora_string = "lora"
    else:
        lora_string = "nolora"
    save_limit = finetune_parameters.get("save_total_limit", -1)
    seq_len = finetune_parameters.get("max_seq_length", -1)
    return (
        model_name
        + "-"
        + "reasoning"
        + "-"
        + f"bs{batch_size}"
        + "-"
        + f"{scheduler}"
        + "-"
        + ds_string
        + "-"
        + lora_string
        + f"-save_limit{save_limit}"
        + f"-seqlen{seq_len}"
    )


pipeline_display_name = get_pipeline_display_name()
print(f"Display name used for the run: {pipeline_display_name}")

In [ ]:
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import Input

# fetch the pipeline component
registry_ml_client = MLClient(credential, registry_name="azureml")
pipeline_component_func = registry_ml_client.components.get(
    name="chat_completion_pipeline", label="latest"
)


# define the pipeline job
@pipeline(name=pipeline_display_name)
def create_pipeline():
    chat_completion_pipeline = pipeline_component_func(
        pytorch_model_path=foundation_model.id,
        compute_model_import=compute_cluster,
        compute_preprocess=compute_cluster,
        compute_finetune=compute_cluster,
        compute_model_evaluation=compute_cluster,
        # map the dataset splits to parameters
        train_file_path=Input(
            type="uri_file", path="./multilingual_thinking_train.jsonl"
        ),
        validation_file_path=Input(
            type="uri_file", path="./multilingual_thinking_valid.jsonl"
        ),
        test_file_path=Input(
            type="uri_file", path="./multilingual_thinking_test.jsonl"
        ),
        # Training settings
        number_of_gpu_to_use_finetuning=gpus_per_node,  # set to the number of GPUs available in the compute
        **finetune_parameters
    )
    return {
        # map the output of the fine tuning job to the output of pipeline job so that we can easily register the fine tuned model
        # registering the model is required to deploy the model to an online or batch endpoint
        "trained_model": chat_completion_pipeline.outputs.mlflow_model_folder
    }


pipeline_object = create_pipeline()

# don't use cached results from previous jobs
pipeline_object.settings.force_rerun = True

# set continue on step failure to False
pipeline_object.settings.continue_on_step_failure = False

Submit the job

In [ ]:
# submit the pipeline job
pipeline_job = workspace_ml_client.jobs.create_or_update(
    pipeline_object, experiment_name=experiment_name
)
# wait for the pipeline job to complete
workspace_ml_client.jobs.stream(pipeline_job.name)

### 6. Register the fine tuned model with the workspace

We will register the model from the output of the fine tuning job. This will track lineage between the fine tuned model and the fine tuning job. The fine tuning job, further, tracks lineage to the foundation model, data and training code.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

# check if the `trained_model` output is available
print("pipeline job outputs: ", workspace_ml_client.jobs.get(pipeline_job.name).outputs)

# fetch the model from pipeline job output - not working, hence fetching from fine tune child job
model_path_from_job = "azureml://jobs/{0}/outputs/{1}".format(
    pipeline_job.name, "trained_model"
)

finetuned_model_name = model_name + "multi-lingual-reasoning"
finetuned_model_name = finetuned_model_name.replace("/", "-")
print("path to register model: ", model_path_from_job)
prepare_to_register_model = Model(
    path=model_path_from_job,
    type=AssetTypes.MLFLOW_MODEL,
    name=finetuned_model_name,
    version=timestamp,  # use timestamp as version to avoid version conflict
    description=model_name
    + " fine tuned model for multi-lingual-thinking chat-completion",
)
print("prepare to register model: \n", prepare_to_register_model)
# register the model from pipeline job output
registered_model = workspace_ml_client.models.create_or_update(
    prepare_to_register_model
)
print("registered model: \n", registered_model)

### 7. Deploy the fine tuned model to an online endpoint
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model.

In [ ]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    ProbeSettings,
    OnlineRequestSettings,
)

# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name

online_endpoint_name = "multi-lingual-reasoning-chat-completion-" + timestamp
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for "
    + registered_model.name
    + ", fine tuned model for multi-lingual-reasoning-chat-completion",
    auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

You can find here the list of SKU's supported for deployment - [Managed online endpoints SKU list](https://learn.microsoft.com/en-us/azure/machine-learning/reference-managed-online-endpoints-vm-sku-list)

In [ ]:
import ast

instance_type = "STANDARD_NC4AS_T4_V3"

# Inference compute allow list that supports deployment
if "inference_compute_allow_list" in foundation_model.tags:
    inference_computes_allow_list = ast.literal_eval(
        foundation_model.tags["inference_compute_allow_list"]
    )  # convert string to python list
    print(f"Please create a compute from the above list - {computes_allow_list}")
else:
    inference_computes_allow_list = None
    print("`inference_compute_allow_list` is not part of model tags")

# Check if the compute is in the allow listed computes
if (
    inference_computes_allow_list is not None
    and instance_type not in inference_computes_allow_list
):
    print(
        f"`instance_type` is not in the allow listed compute. Please select a value from {inference_computes_allow_list}"
    )

# create a deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type=instance_type,
    instance_count=1,
    liveness_probe=ProbeSettings(initial_delay=600),
    request_settings=OnlineRequestSettings(request_timeout_ms=90000),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {"demo": 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

### 8. Test the endpoint with sample data

We will fetch some sample data from the test dataset and submit to online endpoint for inference. We will then show the display the scored labels alongside the ground truth labels

In [ ]:
# read ./multilingual_thinking_test.jsonl into a pandas dataframe
test_df = pd.read_json("./multilingual_thinking_test.jsonl", lines=True)
# take few random samples
test_df = test_df.sample(n=1)
# rebuild index
test_df.reset_index(drop=True, inplace=True)
test_df.head(1)

In [ ]:
import json

# create a json object with the key as "input_data" and value as a list of values from the text column of the test dataframe
parameters = {
    "temperature": 0.6,
    "top_p": 0.9,
    "do_sample": True,
    "max_new_tokens": 200,
}
test_json = {
    "input_data": {
        "input_string": [test_df["messages"][0]],
        "parameters": parameters,
    },
    "params": {},
}
# save the json object to a file named sample_score.json in the ./samsum-dataset folder
with open("./sample_score.json", "w") as f:
    json.dump(test_json, f)

In [ ]:
# score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="demo",
    request_file="./sample_score.json",
)
print("raw response: \n", response, "\n")

### 9. Delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint

In [ ]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()